# Experimento 5 
***
- Rede utilizada: Inception
- Conjunto de Dados: VinBigData
- Modelando uma para classificar problemas pulmonares em 11 classes distintas

### Importação dos pacotes

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.xception import Xception
#from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

### Pré-processamento nos dados

In [2]:
# lendo os dados de um arquivo csv
dataframe = pd.read_csv('/content/drive/MyDrive/vinbigdata/train.csv')
# criando uma coluna com os caminhos relativos as imagens
dataframe['image_path'] = '/content/drive/MyDrive/vinbigdata/train/' + dataframe.image_id + '.jpg'

In [3]:
print('total de imagens disponíveis:', str(len(set(dataframe['image_path']))))

total de imagens disponíveis: 15000


In [4]:
# visualizando os casos disponíveis
dataframe['class_name'].value_counts()

No finding            31818
Aortic enlargement     7162
Cardiomegaly           5427
Pleural thickening     4842
Pulmonary fibrosis     4655
Nodule/Mass            2580
Lung Opacity           2483
Pleural effusion       2476
Other lesion           2203
Infiltration           1247
ILD                    1000
Calcification           960
Consolidation           556
Atelectasis             279
Pneumothorax            226
Name: class_name, dtype: int64

In [5]:
# removendo os casos não relativos a distúrbios pulmonares
dataframe = dataframe[dataframe.class_name != 'Aortic enlargement']
dataframe = dataframe[dataframe.class_name != 'Cardiomegaly']
dataframe = dataframe[dataframe.class_name != 'Other lesion']

In [6]:
# separando os casos rotulados como normais e anormais
normal_cases = dataframe[(dataframe.class_id == 14) & (dataframe.class_name == 'No finding')]
abnormal_cases = dataframe[(dataframe.class_id != 14) & (dataframe.class_name != 'No finding')]

print('total de dados após a filtração:', str(len(set(normal_cases['image_path'])) + len(set(abnormal_cases['image_path']))))

total de dados após a filtração: 13952


In [7]:
# removendo as imagens repetidas
normal_data = normal_cases[['image_path', 'class_name']].drop_duplicates(subset = 'image_path', )
abnormal_data = abnormal_cases[['image_path', 'class_name']].drop_duplicates(subset = 'image_path', )

# criando dataframes especifos com caminhos para as imagens e rótulos
normal_data['target'] = 'normal'
abnormal_data['target'] = 'abnormal'

In [8]:
print('quantidade de dados rotulados como normais:', len(normal_data))
print('quantidade de dados rotulados como anormais:', len(abnormal_data))

quantidade de dados rotulados como normais: 10606
quantidade de dados rotulados como anormais: 3346


In [9]:
# removendo 69% dos casos normais para balancear os dados
normal, _ = train_test_split(normal_data, test_size = 0.69, random_state = 42)

In [10]:
print('quantidade de dados rotulados como normais:', len(normal))
print('quantidade de dados rotulados como anormais:', len(abnormal_data))

quantidade de dados rotulados como normais: 3287
quantidade de dados rotulados como anormais: 3346


In [11]:
# concatenando os dataframes de casos normais e anormais
full_data = abnormal_data

In [12]:
# visualizando a quantidade de exemplos por classe disponíveis
full_data['class_name'].value_counts()

Pleural thickening    901
Pulmonary fibrosis    742
Lung Opacity          427
Nodule/Mass           339
Pleural effusion      328
Calcification         167
Infiltration          163
ILD                   152
Consolidation          59
Atelectasis            41
Pneumothorax           27
Name: class_name, dtype: int64

In [13]:
# importando os dataframes dos dados de treinamento, validação e teste
train_df = pd.read_csv('/content/drive/MyDrive/train_df.csv', sep = ',', index_col=  0)
validation_df = pd.read_csv('/content/drive/MyDrive/validation_df.csv', sep = ',', index_col=  0)
test_df = pd.read_csv('/content/drive/MyDrive/test_df.csv' , sep = ',', index_col=  0)

In [14]:
# tornando as classes na coluna 'labels' categórica
train_df.loc[train_df.labels == 1, 'labels'] = 'abnormal'
train_df.loc[train_df.labels == 0, 'labels'] = 'normal'

validation_df.loc[validation_df.labels == 1, 'labels'] = 'abnormal'
validation_df.loc[validation_df.labels == 0, 'labels'] = 'normal'

test_df.loc[test_df.labels == 1, 'labels'] = 'abnormal'
test_df.loc[test_df.labels == 0, 'labels'] = 'normal'

In [15]:
# separando os casos de anormalidade no dataset nih
train_abnormal_cases = train_df[train_df.labels == 'abnormal']
validation_abnormal_cases = validation_df[validation_df.labels == 'abnormal']
test_abnormal_cases = test_df[test_df.labels == 'abnormal']

In [16]:
# concatenando os dados do dataset nih
nih_data = pd.concat([train_abnormal_cases, validation_abnormal_cases,
                      test_abnormal_cases])

In [17]:
# organizando o nome das colunas
nih_data.rename(columns= {'Image Index': 'image_path', 'finding_labels': 'class_name',
                          'labels': 'target'}, inplace = True)

In [18]:
# concatenando os dados do vinbigdata e do nih
full_data = pd.concat([full_data, nih_data])

In [19]:
# removendo classes ineficientes para o aprendizado da rede 
full_data = full_data[(full_data.class_name != 'ILD') & (full_data.class_name != 'Calcification')]
full_data = full_data[(full_data.class_name != 'Pneumonia') & (full_data.class_name != 'Pleural effusion')]
full_data = full_data[(full_data.class_name != 'Nodule/Mass') & (full_data.class_name != 'Lung Opacity')]
full_data = full_data[full_data.class_name != 'Pleural_Thickening']

In [20]:
# separando os dados de cada uma das classes
infiltration = full_data[full_data.class_name == 'Infiltration']
atelectasis = full_data[full_data.class_name == 'Atelectasis']
nodule = full_data[full_data.class_name == 'Nodule']
pneumothorax = full_data[full_data.class_name == 'Pneumothorax']
consolidation = full_data[full_data.class_name == 'Consolidation']
rest_abnormal_data = full_data[(full_data.class_name == 'Pleural thickening') |
                               (full_data.class_name == 'Emphysema') |
                               (full_data.class_name == 'Pulmonary fibrosis') |
                               (full_data.class_name == 'Fibrosis') |
                               (full_data.class_name == 'Edema')]

In [21]:
# organizando a quantidade de dados disponíveis em cada uma das classes
infiltration_, _ = train_test_split(infiltration, test_size = 0.99, random_state = 42)
atelectasis_, _ = train_test_split(atelectasis, test_size = 0.977, random_state = 42)
nodule_, _ = train_test_split(nodule, test_size = 0.964, random_state = 42)
pneumothorax_, _ = train_test_split(pneumothorax, test_size = 0.956, random_state = 42)
consolidation_, _ = train_test_split(consolidation, test_size = 0.929, random_state = 42)
rest_abnormal_data_, _ = train_test_split(rest_abnormal_data, test_size = 0.88, random_state = 42, 
                                          stratify = rest_abnormal_data['class_name'])
full_data = pd.concat([infiltration_, atelectasis_, nodule_, pneumothorax_, consolidation_, rest_abnormal_data_])

In [22]:
# visualizando o resultado final da quantidade de exemplos disponíveis em cada classe
full_data['class_name'].value_counts()

Pleural thickening    108
Emphysema             107
Nodule                 97
Atelectasis            97
Consolidation          97
Pneumothorax           97
Infiltration           97
Pulmonary fibrosis     89
Fibrosis               87
Edema                  75
Name: class_name, dtype: int64

In [23]:
# separando os dados de treinamento e de teste
train_df, test_df = train_test_split(full_data, stratify = full_data['target'],
                                     test_size = 0.15, random_state = 42)

In [24]:
# separando os dados de validação dos dados de treinamento
train_df, validation_df = train_test_split(train_df, stratify = train_df['target'],
                                           test_size = 0.25, random_state = 42)

In [25]:
# visualizando a quantidade de dados
print('quantidade de imagens de treinamento:', len(train_df['image_path']))
print('quantidade de rótulos de treinamento:', len(train_df['target']))
print('quantidade de imagens de teste:', len(test_df['image_path']))
print('quantidade de rótulos de teste:', len(test_df['target']))
print('quantidade de imagens de validação:', len(validation_df['image_path']))
print('quantidade de rótulos de validação:', len(validation_df['target']))

quantidade de imagens de treinamento: 606
quantidade de rótulos de treinamento: 606
quantidade de imagens de teste: 143
quantidade de rótulos de teste: 143
quantidade de imagens de validação: 202
quantidade de rótulos de validação: 202


In [26]:
# organizando um dicionário para realizar o balanceamento nos dados das classes
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_df['class_name']),
                                                  train_df['class_name'])
class_weight = {0: class_weights[0], 1: class_weights[1], 2: class_weights[2], 3: class_weights[3],
                4: class_weights[4], 5: class_weights[5], 6: class_weights[6], 7: class_weights[7],
                8: class_weights[8], 9: class_weights[9]}

In [27]:
# normalizando as imagens de treinamento e aplicando aumento de dados
image_generator = ImageDataGenerator(rescale = 1./255.,
                                     rotation_range = 10, zoom_range = 0.2)

# criando o gerador de imagens de treinamento 
train_generator = image_generator.flow_from_dataframe(
                                                      dataframe = train_df,
                                                      directory = '',
                                                      x_col = 'image_path',
                                                      y_col = 'class_name',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      target_size = (512, 512))

# normalizando as imagens de teste e validação
test_datagen = ImageDataGenerator(rescale = 1./255.)

# criando o gerador de imagens de validação 
valid_generator = test_datagen.flow_from_dataframe(
                                                      dataframe = validation_df,
                                                      directory = '.', 
                                                      x_col = 'image_path',
                                                      y_col = 'class_name',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      target_size = (512, 512))

test_generator = test_datagen.flow_from_dataframe(
                                                  dataframe = test_df, 
                                                  directory = '.',
                                                  x_col = 'image_path',
                                                  y_col = 'class_name',
                                                  batch_size = 32,
                                                  seed = 42,
                                                  shuffle = True,
                                                  class_mode = 'categorical',
                                                  target_size = (512, 512))

Found 606 validated image filenames belonging to 10 classes.
Found 202 validated image filenames belonging to 10 classes.
Found 143 validated image filenames belonging to 10 classes.


### Preparando a rede neural convolucional

In [28]:
# realizando transferência de aprendizado com a mobile net
model = Xception(input_shape = (512, 512, 3), include_top = False, weights = 'imagenet')

In [29]:
# obtendo a última camada como sendo a nomeada por 'mixed7'
last_layer = model.get_layer('block14_sepconv2_act')
last_output = last_layer.output
#x = layers.GlobalAveragePooling2D()(last_output)
# conectando a rede uma camada de achatamento
x = layers.Flatten()(last_output)
# conectando a rede uma camada com 1024 neurônios e função de ativação relu
x = layers.Dense(units = 512, activation = tf.nn.relu)(x)
# aplicando uma camada de dropout com uma taxa de 20% (normalização)
x = layers.Dropout(rate = 0.2)(x) 
# adicionando uma camada de saída com um neurônio e uma função de ativação sigmoide
x = layers.Dense(units = 10, activation = tf.nn.softmax)(x)      
# conecatando as camadas definidas acima com a arquitetura inception
model = Model(model.input, x) 
# compilando a rede 
model.compile(optimizer = optimizers.RMSprop(learning_rate = 0.0001), loss = 'categorical_crossentropy', 
              metrics = ['acc'])

In [30]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "transferlearning_weights.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')
# definindo um array de callbacks
callbacks = [checkpoint]

In [31]:
# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 606 // 32, 
                              validation_data = valid_generator, validation_steps = 202 // 32,
                              callbacks = callbacks, epochs = 10, class_weight = class_weight)

Epoch 1/10


ResourceExhaustedError: ignored

### Salvando o modelo

In [ ]:
# carregando o melhor peso obtido para o modelo
best_model = model
best_model.load_weights('/content/transferlearning_weights.hdf5')

In [ ]:
# salvando os dois modelos obtidos durante o treinamento
model.save('model1')
best_model.save('model2')

In [ ]:
!mv /content/model1 /content/drive/MyDrive/experimentos/v2.0-exp5-ds4
!mv /content/model2 /content/drive/MyDrive/experimentos/v2.0-exp5-ds4

In [ ]:
# carregando o melhor modelo para realização de testes de desempenho
#model = tf.keras.models.load_model('/content/drive/MyDrive/experimentos/v2.0-exp3-ds4/model2')

In [ ]:
model.evaluate(test_generator)

### Métricas de avaliação do modelo

In [ ]:
# carregando os dados de teste
for i in range(0, 42):
  (x1, y1) = test_generator[i]
  if i == 0:
    x, y = x1, y1
  else:
    x = np.concatenate((x, x1))
    y = np.concatenate((y, y1))

In [ ]:
# realizando a predição para os dados de teste
predict = model.predict(x)

In [ ]:
count_global = 0
for predicts in predict:
    count = 0
    aux = np.zeros((11,))
    for values in predicts:
        if values >= 0.50:
            aux[count] = 1.
        else:
            aux[count] = 0.
        count += 1
    predict[count_global] = aux
    count_global += 1

In [ ]:
print('Matriz de Confusão:\n', confusion_matrix(y.argmax(axis = 1), predict.argmax(axis = 1)))
print('Acurácia:', accuracy_score(y.argmax(axis = 1), predict.argmax(axis = 1)))
print('Precisão', precision_score(y.argmax(axis = 1), predict.argmax(axis = 1)))
print('Sensibilidade:', recall_score(y.argmax(axis = 1), predict.argmax(axis = 1))) 
print('F1_Score:', f1_score(y.argmax(axis = 1), predict.argmax(axis = 1)))

In [ ]:
# visualizando o ganho de acurácia durante o treinamento
plt.figure(figsize = (15,5))
plt.subplot(1, 2, 1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('model-accuracy')

# visualizando o decaimento da função de custo durante o treinamento 
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('model-loss')

### Visualizando a arquitetura da rede

In [ ]:
# visualizando a arquitetura do modelo
model.summary()